In [2]:
from recipe_parser import RecipeConstraintParser
import json
parser = RecipeConstraintParser()


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vjoki\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vjoki\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:

print("SINGLE-TURN PARSING TEST CASES")

single_turn_tests = [
    {
        "input": "Find two vegan dinners under 450 kcal with at least 18 g protein.",
        "expected": {
            "count": 2,
            "diet": ["vegan"],
            "max_calories": 450.0,
            "min_protein": 18.0
        }
    },
    {
        "input": "I need breakfast with protein over 20g, sugar under 10g, in 15 minutes.",
        "expected": {
            "min_protein": 20.0,
            "max_sugar": 10.0,
            "max_duration": 15.0
        }
    },
    {
        "input": "Give me low-carb meals under 30g carbohydrates with protein exceeding 20g.",
        "expected": {
            "diet": ["low-carb"],
            "max_carbs": 30.0,
            "min_protein": 20.0
        }
    },
    {
        "input": "Find recipes with garbanzo beans",
        "expected": {
            "include_ingredients": ["garbanzo bean", "chickpea", "garbanzo", "ceci"]  # synonyms
        }
    },
    {
        "input": "Show dishes containing chickpeas without peanuts",
        "expected": {
            "include_ingredients": ["chickpea", "garbanzo bean", "garbanzo", "ceci"],
            "exclude_ingredients": ["peanut", "groundnut", "goober"]  # synonyms
        }
    },
    {
        "input": "Find meals with protein exceeding 30g and calories below 500",
        "expected": {
            "min_protein": 30.0,
            "max_calories": 500.0
        }
    },
    {
        "input": "I want something with at least 15g protein but under 300 calories",
        "expected": {
            "min_protein": 15.0,
            "max_calories": 300.0
        }
    },
    {
        "input": "Find vegan options with no more than 25g carbs and at least 10g protein",
        "expected": {
            "diet": ["vegan"],
            "max_carbs": 25.0,
            "min_protein": 10.0
        }
    },
    {
        "input": "Show me 3 vegetarian lunches under 400 kcal with less than 600 mg sodium.",
        "expected": {
            "count": 3,
            "diet": ["vegetarian"],
            "max_calories": 400.0,
            "max_sodium": 600.0
        }
    },
    {
        "input": "I want high-protein breakfasts over 25g protein in under 20 minutes.",
        "expected": {
            "min_protein": 25.0,
            "max_duration": 20.0
        }
    },
    {
        "input": "Find desserts under 300 kcal with less than 20g sugar and low saturated fat.",
        "expected": {
            "max_calories": 300.0,
            "max_sugar": 20.0
        }
    },
    {
        "input": "Find dinners that serve 6-8 people with moderate calories.",
        "expected": {
            "min_servings": 6,
            "max_servings": 8
        }
    },
    {
        "input": "Find healthy soup recipes without butter, sodium less than 400mg.",
        "expected": {
            "health_category": ["healthy-2", "healthy"],
            "exclude_ingredients": ["butter"],
            "max_sodium": 400.0
        }
    },
    {
        "input": "Show quick pasta options with chickpeas, under 450 kcal, vegetarian.",
        "expected": {
            "include_ingredients": ["chickpea", "garbanzo bean", "garbanzo", "ceci"],
            "max_calories": 450.0,
            "diet": ["vegetarian"]
        }
    },
    {
        "input": "I want meals with scallions and no peanuts, under 500 calories.",
        "expected": {
            "include_ingredients": ["scallion", "green onion", "spring onion"],
            "exclude_ingredients": ["peanut", "groundnut", "goober"],
            "max_calories": 500.0
        }
    }
]

for i, test in enumerate(single_turn_tests, 1):
    print(f"\n--- Test {i} ---")
    print(f"Input: {test['input']}")
    result = parser.parse(test['input'])
    print(f"Parsed: {json.dumps(result, indent=2)}")
    print(f"Expected keys: {list(test['expected'].keys())}")
    
    # Verify key constraints are present
    missing_keys = []
    for key in test['expected'].keys():
        if key not in result:
            missing_keys.append(key)
    
    if missing_keys:
        print(f"⚠️  MISSING KEYS: {missing_keys}")
    else:
        print("✓ All expected keys found")

print("\n" + "=" * 80)
print("MULTI-TURN CONVERSATION TEST CASES")
print("=" * 80)

multi_turn_tests = [
    {
        "name": "Pasta with calorie/diet preferences",
        "conversation": [
            "Show quick pasta options.",
            "Do you have calorie or diet preferences?",
            "<450 kcal, vegetarian.",
        ],
        "expected": {
            "max_calories": 450.0,
            "diet": ["vegetarian"]
        }
    },
    {
        "name": "Breakfast with time and protein",
        "conversation": [
            "I need breakfast ideas.",
            "What's your time constraint and protein goal?",
            "Under 15 minutes, at least 20g.",
        ],
        "expected": {
            "max_duration": 15.0,
            "min_protein": 20.0
        }
    },
    {
        "name": "Desserts with calorie preference",
        "conversation": [
            "What desserts do you recommend?",
            "Are you looking for something low-calorie or low-sugar?",
            "Low-calorie, under 200 kcal.",
        ],
        "expected": {
            "max_calories": 200.0
        }
    },
    {
        "name": "Chicken with style preference",
        "conversation": [
            "Show me chicken recipes.",
            "Would you prefer grilled, baked, or any specific style?",
            "Something quick and low-carb, under 20g carbs.",
        ],
        "expected": {
            "max_duration": 30.0,  # inferred from "quick"
            "diet": ["low-carb"],
            "max_carbs": 20.0
        }
    },
    {
        "name": "Soup with dietary restrictions",
        "conversation": [
            "I want to make soup.",
            "Any dietary restrictions or sodium concerns?",
            "Yes, low sodium under 400mg and vegetarian.",
        ],
        "expected": {
            "max_sodium": 400.0,
            "diet": ["vegetarian"]
        }
    },
    {
        "name": "Party appetizer with servings",
        "conversation": [
            "I need a party appetizer.",
            "How many people are you serving?",
            "Around 10-12 people.",
        ],
        "expected": {
            "min_servings": 10,
            "max_servings": 12
        }
    }
]

for i, test in enumerate(multi_turn_tests, 1):
    print(f"\n--- Multi-turn Test {i}: {test['name']} ---")
    print("Conversation:")
    for j, msg in enumerate(test['conversation']):
        role = "User" if j % 2 == 0 else "Assistant"
        print(f"  {role}: {msg}")
    
    result = parser.parse_conversation(test['conversation'])
    print(f"\nParsed constraints: {json.dumps(result, indent=2)}")
    print(f"Expected keys: {list(test['expected'].keys())}")
    
    missing_keys = []
    incorrect_values = []
    
    for key, expected_value in test['expected'].items():
        if key not in result:
            missing_keys.append(key)
        elif isinstance(expected_value, list):
            if not all(item in result.get(key, []) for item in expected_value):
                incorrect_values.append(f"{key}: expected {expected_value}, got {result.get(key)}")
        elif isinstance(expected_value, (int, float)):
            if abs(result.get(key, 0) - expected_value) > 0.01:
                incorrect_values.append(f"{key}: expected {expected_value}, got {result.get(key)}")
    
    if missing_keys:
        print(f"MISSING KEYS: {missing_keys}")
    if incorrect_values:
        print(f"INCORRECT VALUES: {incorrect_values}")
    if not missing_keys and not incorrect_values:
        print("All expected constraints matched correctly")

SINGLE-TURN PARSING TEST CASES

--- Test 1 ---
Input: Find two vegan dinners under 450 kcal with at least 18 g protein.
Parsed: {
  "count": 2,
  "max_calories": 450.0,
  "min_protein": 18.0,
  "diet": [
    "vegan"
  ]
}
Expected keys: ['count', 'diet', 'max_calories', 'min_protein']
✓ All expected keys found

--- Test 2 ---
Input: I need breakfast with protein over 20g, sugar under 10g, in 15 minutes.
Parsed: {
  "min_protein": 20.0,
  "max_sugar": 10.0,
  "max_duration": 15.0
}
Expected keys: ['min_protein', 'max_sugar', 'max_duration']
✓ All expected keys found

--- Test 3 ---
Input: Give me low-carb meals under 30g carbohydrates with protein exceeding 20g.
Parsed: {
  "max_carbs": 30.0,
  "min_protein": 20.0,
  "diet": [
    "low-carb"
  ]
}
Expected keys: ['diet', 'max_carbs', 'min_protein']
✓ All expected keys found

--- Test 4 ---
Input: Find recipes with garbanzo beans
Parsed: {
  "include_ingredients": [
    "bean",
    "bean plant",
    "beans",
    "chickpea",
    "edible b